# Plotting of DMS and Chlorophyll-A for CMAQ

---
    author: Barron H. Henderson
    date: 2021-03-23
    last updated: 2022-04-29
    contributors: Brett Gantt, Jeff Willison, and Golam Sarwar
---

This notebook creates figures from DMS and CHLO from OCEAN files and the files used to create them. The inputs are often only viewed to diagnose unexpected results, as are the intermediate files. The visualization notebook will only work if the results are already available. In cloud-based systems, this may require uploading the files.



# Specify User Input Options

* User input options are described below.
* Most users will update `dom`'
* `inline` is set to false to avoid saving figures in repository, set inline to show figures in addition to saving them.

In [1]:
# dom : str
#     Name of output domain. For example, 12US1, 36US3, 108NHEMI2. This is used
#     to name outputs and inputs.
dom = '12US1'

# inline : bool
#     Force display of plots interactively in the notebook. Otherwise, use Agg
inline = False

In [2]:
# CMAQ-ready Files
ocnouttmpl = f'output/{dom}/OCEAN_*_L3m*.nc'
# Intermediate Files
dmsoutpath = f'dmsclimatology/{dom}/dmsconcentration.{dom}.nc'
chlotmpl = f'chlor_a/{dom}/A*.nc'


# Import libraries


In [3]:
if inline:
    %matplotlib inline
else:
    from matplotlib import use
    # If Agg is not available, comment out
    use('Agg')

In [4]:
import os
from copy import copy
from glob import glob
import warnings

import numpy as np
import matplotlib.pyplot as plt
import pyproj

import pycno
import PseudoNetCDF as pnc

from IPython.display import clear_output, display

os.environ['IOAPI_ISPH'] = '6370000.'
warnings.simplefilter('ignore')

In [5]:
os.makedirs(f'figs/native/', exist_ok=True)
os.makedirs(f'figs/{dom}/', exist_ok=True)

# Visualize DMS and CHLO in CMAQ-ready File

* If working in the cloud, make sure the files have been uploaded.

In [6]:
ocnoutpaths = sorted(glob(ocnouttmpl))

In [7]:
gdpath = ocnoutpaths[0]
gdf = pnc.pncopen(gdpath, format='ioapi')
gproj = gdf.getproj(withgrid=True)
gcno = pycno.cno(proj=gproj)
cno = pycno.cno()


In [8]:
cmap = copy(plt.get_cmap('viridis'))
cmap.set_under('grey')
chlonorm = plt.Normalize(vmin=1e-6, vmax=4)
dmsnorm = plt.Normalize(vmin=1e-6, vmax=8)

In [9]:
for ocnoutpath in ocnoutpaths:
    figpath = os.path.join('figs', dom, os.path.basename(ocnoutpath)) + '.png'
    ocnf = pnc.pncopen(ocnoutpath, format='ioapi')
    fig, axx = plt.subplots(1, 2, figsize=(7, 4))
    p = axx[0].pcolormesh(ocnf.variables['CHLO'][0, 0], norm=chlonorm, cmap=cmap)
    fig.colorbar(p, orientation='horizontal', ax=axx[0], label='CHLO mg/m3', extend='min');
    p = axx[1].pcolormesh(ocnf.variables['DMS'][0, 0], norm=dmsnorm, cmap=cmap)
    fig.colorbar(p, orientation='horizontal', ax=axx[1], label='DMS nM', extend='min');
    fig.suptitle(ocnoutpath)
    for ax in axx.ravel():
        gcno.draw(ax=ax)
    fig.savefig(figpath)

# Intermediate File Visualization for Debugging

* The cells below require intermediate files.
* If you are in the cloud, these files may not be present. 

## DMS Processing Vizualization

* Vizualize lat/lon DMS
* Vizualize the gridded DMS

In [10]:
dmsfile = pnc.pncopen('dmsclimatology/dmsconcentration.nc', format='netcdf')
times = dmsfile.getTimes()
lon = dmsfile.variables['longitude'][:]
lat = dmsfile.variables['latitude'][:]
dmsvar = dmsfile.variables['DMS']
fig, axx = plt.subplots(4, 3, figsize=(10, 8), sharex=True, sharey=True)
cax = fig.add_axes([.1, .1, .8, .025])
for ai, ax in enumerate(axx.ravel()):
    p = ax.pcolormesh(lon, lat, dmsvar[ai], norm=dmsnorm, shading='nearest')
    fig.colorbar(p, cax=cax, orientation='horizontal')
    ax.set_title(times[ai].strftime('%Y-%m-%d'))
    cno.draw(ax=ax)

fig.savefig('figs/native/dmsconcentration.nc.png')

## Visualize Interpolated DMS Concentrations

* Overland portions of the domain are masked before making files CMAQ-ready.

In [11]:
dmsdomfile = pnc.pncopen(dmsoutpath, format='netcdf')
dmsfigpath = os.path.join('figs', dom, os.path.basename(dmsoutpath) + '.png')
dmsvar = dmsdomfile.variables['DMS']
times = dmsdomfile.getTimes()
fig, axx = plt.subplots(4, 3, figsize=(10, 10), sharex=True, sharey=True)
cax = fig.add_axes([.1, .075, .8, .025])
lnorm = plt.matplotlib.colors.LogNorm(vmin=1, vmax=50)

for ai, ax in enumerate(axx.ravel()):
    p = ax.pcolormesh(dmsvar[ai], norm=dmsnorm)
    fig.colorbar(p, cax=cax, orientation='horizontal',)
    ax.set_title(times[ai].strftime('%Y-%m-%d'))
    gcno.draw(ax=ax)

fig.savefig(dmsfigpath)

## Chlor-A Processing Vizualization

* Vizualize lat/lon Chlor-A
* Vizualize the interpolated gridded Chlor-A

In [12]:
chlinpaths = sorted(glob('chlor_a/A*.nc'), key=lambda x: os.path.basename(x)[5:])
chlorfs = [pnc.pncopen(tmppath).insertDimension(time=1, before='lat', multionly=True) for tmppath in  chlinpaths]
chlorf = chlorfs[0].stack(chlorfs[1:], 'time')
lat = chlorf.variables['lat']
lon = chlorf.variables['lon']
chlorvar = chlorf.variables['chlor_a']

fig, axx = plt.subplots(4, 3, figsize=(10, 8), sharex=True, sharey=True, dpi=144)
cax = fig.add_axes([.1, .1, .8, .025])
nthin = 1

lnorm = plt.matplotlib.colors.LogNorm()
for ai, ax in enumerate(axx.ravel()):
    p = ax.pcolormesh(
        lon[::nthin], lat[::nthin], chlorvar[ai, ::nthin, ::nthin],
        norm=chlonorm, shading='nearest'
    )
    fig.colorbar(p, cax=cax, orientation='horizontal')
    ax.set_title(times[ai].strftime('%Y-%m-%d'))
    cno.draw(ax=ax)

fig.savefig('figs/native/chlor_a.png')

* Note: that overland concentrations will be masked in the CMAQ-ready files.

In [13]:
chlornewgridpaths = sorted(glob(chlotmpl), key=lambda x: os.path.basename(x)[5:])
chlordomfs = [pnc.pncopen(tmppath).insertDimension(TSTEP=1, before='ROW', multionly=True) for tmppath in  chlornewgridpaths]
chlordomf = chlordomfs[0].stack(chlordomfs[1:], 'TSTEP')
chlorvar = chlordomf.variables['chlor_a']

fig, axx = plt.subplots(4, 3, figsize=(10, 10), sharex=True, sharey=True)
cax = fig.add_axes([.1, .075, .8, .025])

for ai, ax in enumerate(axx.ravel()):
    p = ax.pcolormesh(chlorvar[ai], norm=chlonorm)
    ax.set_title(times[ai].strftime('%Y-%m-%d'))
    gcno.draw(ax=ax)
    
fig.colorbar(p, cax=cax, orientation='horizontal')
fig.savefig(f'figs/{dom}/chlor_a.{dom}.png')